In [10]:
import pandas as pd
import os 
import numpy as np
import matplotlib.pyplot as plt


In [3]:
!pwd
os.chdir('../')
!pwd

/home/yukikongju/Projects/tidytuesday/bixi/notebooks
/home/yukikongju/Projects/tidytuesday/bixi


In [27]:
# yearly capacity for given station_code
station_code = '7022'
capacity_raw_dir = 'data/capacity_raw/2018/'


# get stations csv files name
station_file_paths = []
for subdir, dirs, files in os.walk(capacity_raw_dir):
    for file in files:
        if station_code in file: 
            csv_path = os.path.join(subdir, file)
            station_file_paths.append(csv_path)

# concatenate csv files
dfs = []
for file in station_file_paths:
    df = pd.read_csv(file)
    dfs.append(df)
df_concat = pd.concat(dfs).drop(columns=['index'])

# sort by timestamp
df_concat = df_concat.sort_values(by=['timestamp'])

In [28]:
df_concat

,timestamp,station_code,END,START
0,2018-04-10 18:30:00,7022,1,0
1,2018-04-10 19:55:00,7022,1,0
2,2018-04-10 21:30:00,7022,1,0
3,2018-04-10 21:35:00,7022,1,0
0,2018-04-11 00:00:00,7022,0,1
...,...,...,...,...
14,2018-11-15 20:15:00,7022,1,0
15,2018-11-15 22:45:00,7022,0,1
16,2018-11-15 22:55:00,7022,0,1
17,2018-11-15 23:35:00,7022,0,1


In [33]:
# station capacity
df_stations = pd.read_csv('data/stations.csv')
station_capacity = df_stations[df_stations['short_name'] == station_code]['capacity']
station_capacity

484    15
Name: capacity, dtype: int64

In [49]:
# compute yearly station capacity
# assumption: every year, start at max capacity (might be wrong!)
# PROBLEM: why are the first event END ???

capacity = int(station_capacity)
capacity_count = []
for _, row in df_concat.iterrows():
    capacity = capacity + row['END'] - row['START']
    capacity_count.append(capacity)
df_concat['capacity'] = capacity_count


In [54]:
df_concat[1000:10010]
# df_concat[:20]

,timestamp,station_code,END,START,capacity
10,2018-05-15 13:50:00,7022,1,0,-179
11,2018-05-15 13:55:00,7022,0,1,-180
12,2018-05-15 14:50:00,7022,1,0,-179
13,2018-05-15 16:30:00,7022,0,1,-180
14,2018-05-15 16:45:00,7022,1,0,-179
...,...,...,...,...,...
14,2018-11-15 20:15:00,7022,1,0,-1270
15,2018-11-15 22:45:00,7022,0,1,-1271
16,2018-11-15 22:55:00,7022,0,1,-1272
17,2018-11-15 23:35:00,7022,0,1,-1273


In [60]:
pd.set_option('display.max_rows', None)
print(df_concat)

              timestamp  station_code  END  START  capacity
0   2018-04-10 18:30:00          7022    1      0        16
1   2018-04-10 19:55:00          7022    1      0        17
2   2018-04-10 21:30:00          7022    1      0        18
3   2018-04-10 21:35:00          7022    1      0        19
0   2018-04-11 00:00:00          7022    0      1        18
1   2018-04-11 08:50:00          7022    0      3        15
2   2018-04-11 09:30:00          7022    0      1        14
3   2018-04-11 10:30:00          7022    1      0        15
4   2018-04-11 11:15:00          7022    0      1        14
5   2018-04-11 11:20:00          7022    0      1        13
6   2018-04-11 13:05:00          7022    0      1        12
7   2018-04-11 14:35:00          7022    0      1        11
8   2018-04-11 14:45:00          7022    1      0        12
9   2018-04-11 16:15:00          7022    0      1        11
10  2018-04-11 16:50:00          7022    1      0        12
11  2018-04-11 18:40:00          7022   

In [53]:
df_concat[df_concat['capacity'] < 0]

,timestamp,station_code,END,START,capacity
3,2018-04-19 09:55:00,7022,0,1,-1
4,2018-04-19 10:40:00,7022,0,1,-2
5,2018-04-19 11:40:00,7022,1,0,-1
6,2018-04-19 12:00:00,7022,0,1,-2
7,2018-04-19 13:15:00,7022,1,1,-2
...,...,...,...,...,...
14,2018-11-15 20:15:00,7022,1,0,-1270
15,2018-11-15 22:45:00,7022,0,1,-1271
16,2018-11-15 22:55:00,7022,0,1,-1272
17,2018-11-15 23:35:00,7022,0,1,-1273
